In [240]:
import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Main goal: 
Fit a model for multisensory localization as a weighted average of visual and auditory information in a first round of experiments, then test it in a second round of experiment. The underlying assumption is that the mulitmodal localization can be modelled as some weighted average of the purely auditory/visual inputs via:

$$ L^* = w_v L^*_v +  w_a L^*_a  $$

where 
$w_v, w_a$  are the weights for the auditory/visual input, 
$L^*_w, L^*_a$ are the location estimates based on the purely visual/auditory input and 
$L^*$ is the location estimate based on the combined input.

## First round of experiments: purely visual & purely auditory
__Goals__:
- estimate weights $w_a, w_v$ for the auditory and visual information in the model of the multisensory setup
- find location estimates $L_a^*, L_v^*$ based on the purely auditory/visual inputs 

__Method__:
This will be achieved via  fitting a cumulative normal distribution to fit the datapoints we see - using the method of *Maximum Likelihood Estimation* (MLE). Recall that a normal distribution is defined by it's variance $\sigma^2$ and it's mean $\mu$. We thus want to estimate 

$$ \widehat{\sigma}^2, \widehat{\mu} = argmax_{\mu, \sigma} \prod_{t=1}^{T} p_t^{r_t} (1-p_t)^{1-r_t}
= argmax_{\mu, \sigma} \prod_{t=1}^{T} (\frac{1}{2}(1+erf(\frac{x-\mu}{\sqrt(2\sigma^2)})) )^{r_t} (1-(1+erf(\frac{x-\mu}{\sqrt(2\sigma^2)})))^{1-r_t}
$$

*TODO: annoying work of explaining all the variables and why a Bernoulli probability is the right choice*

Battaglia et al. suggested the following modification to the "classical MLE" model: transform it into a Bayesian model by adding priors $p(\sigma)$ and $p(\mu)$. The reason is the (evolutionary?) bias towards visual input as a more reliable source that the MLE model doesn't capture. In fact, some authors claim that this visual dominance is even so strong that the visual sensory input will completely dominate the auditory one (*visual capture theroy*). We thus refine our model into 

## this is not correct

$$ \widehat{\sigma}^2, \widehat{\mu} = argmax_{\mu, \sigma} \prod_{t=1}^{T} (\frac{1}{2}(1-erf(\frac{x-\mu}{\sqrt(2\sigma^2)})) )^{r_t} (1-(1-erf(\frac{x-\mu}{\sqrt(2\sigma^2)})))^{1-r_t} \cdot p_{\sigma^2}(\sigma^2) \cdot p_{\mu}(\mu)$$

The prior for $\mu$ is just a uniform distribution. More interestingly, the prior for $\sigma^2$ is an *inverse gamma distribution* that we model in a way that it favors small variances (corresponding to __reliable__ sensory input) 
[*TODO: explain why and how this is, write out mathzzz*].

needed python functionalities:
    - argmax / maximum likelihood
    - inverse gamma distribution
    - uniform distribution
    
possibly useful:
    - stats.bernoulli

In [60]:
from scipy import stats, optimize
from math import erf, sqrt, exp

In [88]:
def phi(x, mu, sigma_sq):
    distribution = stats.norm(loc = mu, scale = sigma_sq)
    return distribution.cdf(x)

In [91]:
phi(-6,0,5)

0.11506967022170822

In [306]:
igdistr = stats.invgamma(a=46, scale =1e-34)

In [315]:
igdistr.cdf(1)

1.0

In [295]:
def round1_likelihood(degrees, answers, mu, sigma_sq):
# likelihood(R|mu, sigma) = 
    likelihood = 1
    # model the product
    for t in range(len(degrees)):
        #print('prob here', stats.bernoulli.pmf(answers[t], phi(degrees[t], mu, sigma_sq)))
        likelihood = likelihood * stats.bernoulli.pmf(answers[t], phi(degrees[t], mu, sigma_sq), loc=0)
        #print(likelihood)
        #stats.uniform.pdf(mu)
        #*p(mu)*p(sigma_sq) - i.e. go from MLE to Bayesian approach
    return likelihood

In [290]:
def compute_MLE_1(data):
    degrees = data[0, :]
    answers = data[1, :]
    result = optimize.minimize(lambda x: 1-round1_likelihood(degrees, answers, *x), x0 = [0,2] , tol = 1e-10)['x']
    mu = result[0]
    sigma2 = result[1]
    return {'mu':mu, 'sigma2':sigma2}

In [291]:
# get the weights from the sigmas we calculated
def get_weight_from_variance(variance, other_variance):
    return (1/variance)/((1/variance + 1/other_variance))

wfv_vectorized = np.vectorize(get_weight_from_variance)

### Apply to our data - audio

In [292]:
all_participants_audio = []
for filename in os.listdir('data/preprocessed/audio'):
    if filename.endswith('.p'):
        print(filename)
        data = pickle.load( open('data/preprocessed/audio/' + filename, "rb" ) )
        #print(data[1, :])
        results = compute_MLE_1(data)
        results['participant'] = filename[:-2]
        all_participants_audio.append(results)

participant1.p
participant10.p
participant2.p
participant3.p
participant4.p
participant5.p
participant6.p
participant8.p
participant9.p


In [274]:
audio_results = pd.DataFrame(all_participants_audio)
audio_results[audio_results['mu'] > 0]

,mu,participant,sigma2
0,0.400999,participant1,1.883169
2,1.383120,participant2,2.675663
3,2.319435,participant3,1.590689
4,1.336872,participant4,2.173871
5,0.651118,participant5,2.390670
6,1.124995,participant6,1.243926
8,0.151848,participant9,1.833054


### - video

In [182]:
all_participants = []
for filename in os.listdir('data/preprocessed/video'):
    if filename.endswith('.p'):
        print(filename)
        data = pickle.load( open('data/preprocessed/video/' + filename, "rb" ) )
        for noise in data.keys():
            data_here = data[noise]
            print(data_here.shape)
            results = compute_MLE_1(data_here)
            results['noise'] = noise
            results['participant'] = filename[:-2]
            all_participants.append(results)

participant1.p
(2, 70)
(2, 70)
(2, 70)
(2, 70)
(2, 70)
participant10.p
(2, 70)
(2, 70)
(2, 70)
(2, 70)
(2, 70)
participant2.p
(2, 70)
(2, 70)
(2, 70)
(2, 70)
(2, 70)
participant3.p
(2, 70)
(2, 70)
(2, 70)
(2, 70)
(2, 70)
participant4.p
(2, 70)
(2, 70)
(2, 70)
(2, 70)
(2, 70)
participant5.p
(2, 70)
(2, 70)
(2, 70)
(2, 70)
(2, 70)
participant6.p
(2, 70)
(2, 70)
(2, 70)
(2, 70)
(2, 70)
participant8.p
(2, 70)
(2, 70)
(2, 70)
(2, 70)
(2, 70)
participant9.p
(2, 70)
(2, 70)
(2, 70)
(2, 70)
(2, 70)


In [293]:
video_results = pd.DataFrame(all_participants)

In [294]:
final_video_results = video_results.groupby(['participant','noise']).mean()
final_video_results[final_video_results['mu']>0]

mu       sigma2
participant   noise                          
participant1  0.10      0.017520     2.101098
              0.23      1.246649     1.475694
participant10 0.10      0.458920     2.119718
participant2  0.10      0.189812     1.833054
              0.23      0.189812     1.833054
              0.36      0.151848     1.833054
              0.49      0.189812     1.833054
              0.62      0.942930     2.514325
participant3  0.10   9451.321648 -4738.190597
              0.23      0.113892     1.833054
              0.36      0.151848     1.833054
              0.49      0.151848     1.833054
participant4  0.10      0.075928     1.833054
              0.23      0.113892     1.833054
              0.36      0.075928     1.833054
              0.49      0.075928     1.833054
              0.62      0.923703     2.125959
participant5  0.10      0.595734     1.602465
              0.23   9451.321648 -4738.190597
              0.36      1.003304     1.475720
              0.49      0.037964     1.833054
participant6  0.10      0.189812     1.833054
              0.23      0.189812     1.833054
              0.36      0.075928     1.833054
              0.49      1.246649     1.475694
participant8  0.10      0.189812     1.833054
              0.23      0.189812     1.833054
              0.36      0.151848     1.833054
              0.49      1.246649     1.475694
              0.62      0.113892     1.833054
participant9  0.10      0.189812     1.833054
              0.23      0.189812     1.833054
              0.36      0.189812     1.833054
              0.62      0.113892     1.833054

In [271]:
final_video_results['visual_weights'] = wfv_vectorized(final_video_results['sigma2'].as_matrix(), np.repeat(audio_results['sigma2'].as_matrix(), 5))
final_video_results

mu       sigma2  visual_weights
participant   noise                                          
participant1  0.10      0.017520     2.101098        0.472651
              0.23      1.246649     1.475694        0.560657
              0.36      0.000000     2.000000        0.484957
              0.49      0.000000     2.000000        0.484957
              0.62      0.000000     2.000000        0.484957
participant10 0.10      0.458920     2.119718        0.485470
              0.23     -0.126920     1.362624        0.594774
              0.36     -0.569564     1.781921        0.528832
              0.49      0.000000     2.000000        0.500000
              0.62      0.000000     2.000000        0.500000
participant2  0.10      0.189812     1.833054        0.593442
              0.23      0.189812     1.833054        0.593442
              0.36      0.151848     1.833054        0.593442
              0.49      0.189812     1.833054        0.593442
              0.62      0.942930     2.514325        0.515543
participant3  0.10   9451.321648 -4738.190597       -0.000336
              0.23      0.113892     1.833054        0.464605
              0.36      0.151848     1.833054        0.464605
              0.49      0.151848     1.833054        0.464605
              0.62     -0.129517     1.864007        0.460442
participant4  0.10      0.075928     1.833054        0.542529
              0.23      0.113892     1.833054        0.542529
              0.36      0.075928     1.833054        0.542529
              0.49      0.075928     1.833054        0.542529
              0.62      0.923703     2.125959        0.505571
participant5  0.10      0.595734     1.602465        0.598695
              0.23   9451.321648 -4738.190597       -0.000505
              0.36      1.003304     1.475720        0.618321
              0.49      0.037964     1.833054        0.566010
              0.62      0.000000     2.000000        0.544489
participant6  0.10      0.189812     1.833054        0.404268
              0.23      0.189812     1.833054        0.404268
              0.36      0.075928     1.833054        0.404268
              0.49      1.246649     1.475694        0.457389
              0.62      0.000000     2.000000        0.383463
participant8  0.10      0.189812     1.833054        0.521777
              0.23      0.189812     1.833054        0.521777
              0.36      0.151848     1.833054        0.521777
              0.49      1.246649     1.475694        0.575425
              0.62      0.113892     1.833054        0.521777
participant9  0.10      0.189812     1.833054        0.500000
              0.23      0.189812     1.833054        0.500000
              0.36      0.189812     1.833054        0.500000
              0.49      0.000000     2.000000        0.478223
              0.62      0.113892     1.833054        0.500000

## Second round of experiments:
Compare the weights obtained in the second experiment (and thus the model for multisensory integration) and compare it with empirical results. The empirical weights are again found via a maximum likelihood estimation similar to the monosensory trials, but with a modified probability $p_t$:
\begin{align}
\widehat{w}_a, \widehat{w}_v &= argmax_{w_a, w_v} \prod_{t=1}^{T} p_t^{r_t} (1-p_t)^{1-r_t} \\
&= argmax_{w_a, w_v} \prod_{t=1}^{T} (\frac{1}{1 + exp[-(L_c - L_s)/\tau]})^{r_t} (1-(\frac{1}{1 + exp[-(L_c - L_s)/\tau])}^{1-r_t}) \\
&= argmax_{w_a, w_v} \prod_{t=1}^{T} (\frac{1}{1 + exp[-(w_vL_v^c + w_aL_a^c - (w_vL_v^s + w_aL_a^s))/\tau]})^{r_t} (1-(\frac{1}{1 + exp[-(w_vL_v^c + w_aL_a^c - (w_vL_v^s + w_aL_a^s)))/\tau])}^{1-r_t})
\end{align}

*NOTE: I am still ab bit confused about their explanation with 'location estimates' here. I think we probably just have to use the actual locations because we don't really have any location estimates other than the mean of the two distributions fitted in the first round which should just be very close to zero. Is this just a mistake in the paper?
Related question: the visual and audio location is the same in the comparison stimulus then, right? Then L_c collapses  (since the weights sum to 1) Also it makes no sense to optimize for both of the weights since they are constrained to sum to one.*
With these assumptions the equation would simplify to 

$$
argmax_{w_a, w_v} \prod_{t=1}^{T} (\frac{1}{1 + exp[-(L_c - (w_vL_v^s + (1-w_v)L_a^s))/\tau]})^{r_t} (1-(\frac{1}{1 + exp[-(L_c - (w_vL_v^s + (1-w_v)L_a^s)))/\tau])}^{1-r_t}).
$$

In [227]:
# rewrite w_a = 1 - w_v
def round2_likelihood(degrees, answers, w_v, tau):
    l_s_a = 2.25
    l_s_v = -2.25
    # adjusted paper locations to our 7-location setting - is this CORRECT?
    likelihood = 1
    # model the product
    for t in range(len(degrees)):
        #likelihood = (likelihood * 1 / (1 + exp(- (L[t] - w_v*l_s_v + w_a*l_s_a))/tau)**answers[t] 
        #                        * ( 1 - 1 / (1 + exp(- (L[t] - w_v*l_s_v + w_a*l_s_a))/tau))**(1 - answers[t]))
        # rewrite w_a = 1 - w_v
        likelihood = likelihood * (1 / (1 + exp(- (degrees[t] - (w_v*l_s_v + (1-w_v)*l_s_a))/tau))**answers[t] 
                                * ( 1 - (1 / (1 + exp(- (degrees[t] - w_v*l_s_v + (1-w_v)*l_s_a))/tau)))**(1 - answers[t]))
    return likelihood

In [279]:
def compute_MLE_2(data):
    degrees = data[0, :]
    print(degrees)
    answers = data[1, :]
    print(answers)
    result = optimize.minimize(lambda x: 1-round2_likelihood(degrees, answers, *x), 
                               [0.200000, 1.000000], method = 'L-BFGS-B' , bounds = [(0, 1),(None, None)], tol =1e-10)['x']
    w_a = result[0]
    tau = result[1]
    return {'w_a':w_a, 'tau':tau}

### Apply to our data - combined

In [280]:
all_participants_combined = []
for filename in os.listdir('data/preprocessed/combined'):
    if filename.endswith('.p'):
        print(filename)
        data = pickle.load( open('data/preprocessed/combined/' + filename, "rb" ) )
        for noise in data.keys():
            data_here = data[noise]
            print(data_here.shape)
            results = compute_MLE_2(data_here)
            results['noise'] = noise
            results['participant'] = filename[:-2]
            all_participants_combined.append(results)

participant1.p
(2, 70)
[-2.25 -6.75  6.75  6.75 -2.25  4.5   4.5  -4.5   0.   -6.75  6.75  2.25
  4.5   6.75  4.5   0.   -6.75  0.    6.75  2.25  4.5   6.75 -4.5   0.
 -2.25  4.5   0.   -6.75 -4.5   0.   -4.5   6.75  2.25  2.25  0.    2.25
 -4.5  -4.5  -2.25 -6.75  6.75 -6.75 -6.75  2.25  6.75  2.25  2.25 -2.25
 -6.75  0.    2.25 -2.25  4.5  -4.5   0.   -4.5  -2.25 -4.5  -6.75  6.75
 -2.25 -6.75  4.5  -2.25  4.5   0.    2.25  4.5  -4.5  -2.25]
[ 0.  0.  1.  1.  0.  1.  1.  0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  0.
  1.  1.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  1.  0.  0.  1.
  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  1.  1.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  0.  1.  0.  0.  1.  0.  1.  0.  0.  1.  0.  0.]
(2, 70)
[-4.5   6.75 -6.75  2.25 -4.5   0.   -6.75  6.75 -2.25 -2.25  6.75  2.25
  6.75  4.5   0.    2.25  0.    2.25 -6.75  0.   -6.75 -6.75  2.25 -2.25
  6.75  2.25  2.25  4.5   4.5  -2.25  4.5   4.5  -4.5  -2.25 -2.25 -4.5   0.
 -6.75 -2.25 -4.5  -2.25  6.75  0.  

In [281]:
all_participants_combined

[{'noise': 0.1,
  'participant': 'participant1',
  'tau': 1.0,
  'w_a': 0.20000000000000001},
 {'noise': 0.23,
  'participant': 'participant1',
  'tau': 1.0,
  'w_a': 0.20000000000000001},
 {'noise': 0.62,
  'participant': 'participant1',
  'tau': 1.0,
  'w_a': 0.20000000000000001},
 {'noise': 0.49,
  'participant': 'participant1',
  'tau': 1.0,
  'w_a': 0.20000000000000001},
 {'noise': 0.36,
  'participant': 'participant1',
  'tau': 1.0,
  'w_a': 0.20000000000000001},
 {'noise': 0.1,
  'participant': 'participant10',
  'tau': 1.0,
  'w_a': 0.20000000000000001},
 {'noise': 0.23,
  'participant': 'participant10',
  'tau': 1.0,
  'w_a': 0.20000000000000001},
 {'noise': 0.62,
  'participant': 'participant10',
  'tau': 1.0,
  'w_a': 0.20000000000000001},
 {'noise': 0.49,
  'participant': 'participant10',
  'tau': 1.0,
  'w_a': 0.20000000000000001},
 {'noise': 0.36,
  'participant': 'participant10',
  'tau': 1.0,
  'w_a': 0.20000000000000001},
 {'noise': 0.1,
  'participant': 'participant2'